In [1]:
import numpy as np
from numpy.linalg import norm
import pandas as pd
from gensim.models import KeyedVectors
from datetime import datetime
import bert_score
from custom_score.score import score
from custom_score.utils import model_load
import pickle

c:\Users\orteg\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\orteg\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\Users\orteg\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
C:\Users\orteg\AppData\Roaming\Python\Python38\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## I - Custom BERTScore Build

In [8]:
ref = ["Mon prenom est marius"]
can = ["Je suis marius"]

### 1 - Token Representation

In [14]:
def model_load(model):
    assert(type(model) == str)
    if model == "Word2Vec":
        wordvector_path = r'D:\COURS\A4\S8\Stage\Documents\Supervised-Learning-using-Unsupervised-Learning-Metrics-in-the-absence-of-Annotated-Data\custom_BERTScore\GoogleNews-vectors-negative300.bin.gz'
        emb = KeyedVectors.load_word2vec_format(wordvector_path, binary=True)
    if model == "Glove":
        glove_path = r'D:\COURS\A4\S8\Stage\Documents\Supervised-Learning-using-Unsupervised-Learning-Metrics-in-the-absence-of-Annotated-Data\custom_BERTScore\glove2word2vec.txt'
        emb = KeyedVectors.load_word2vec_format(glove_path)
    else:
        print("Model not currently supported")
    return emb

In [15]:
def encode(corpus, model):
    encoded_corpus = []
    unknown = 0
    for sentence in corpus:
        encoded_sentence = []
        for word in sentence.split(" "):
            try:
                encoded_sentence.append(model[word])
            except:
                unknown += 1
        encoded_corpus.append(encoded_sentence)
    return np.array(encoded_corpus, dtype=object), unknown

##### Word2Vec

In [7]:
w2v = model_load("Word2Vec")

In [24]:
#references, n_unknown_ref = encode(["I am Marius", "I like trains"], w2v)
#candidates, n_unknown_cand = encode(["My name is Marius", "I enjoy rail vehicules"], w2v)

references, n_unknown_ref = encode(["I am Marius"], w2v)
candidates, n_unknown_cand = encode(["My name is Marius"], w2v)

In [25]:
print("Shape Reference : ", references.shape, "||", "Unknown Token Reference : ", n_unknown_ref)
print("Shape Candidate : ", candidates.shape, "||", "Unknown Token Candidate : ", n_unknown_cand)

Shape Reference :  (1, 3, 300) || Unknown Token Reference :  0
Shape Candidate :  (1, 4, 300) || Unknown Token Candidate :  0


##### Fasttext

Conversition to word2vec format (Do not compile unless necessary)

In [50]:
import fasttext

In [ ]:
ft_model = fasttext.load_model(r'D:\COURS\A4\S8\Stage\Documents\Supervised-Learning-using-Unsupervised-Learning-Metrics-in-the-absence-of-Annotated-Data\custom_BERTScore\wiki.en\wiki.en.bin')
kv = KeyedVectors(vector_size=ft_model.get_dimension())
all_words = ft_model.get_words()
all_vectors = [ft_model.get_word_vector(w) for w in all_words]
kv.add_vectors(all_words, all_vectors)
kv.save_word2vec_format('ftwords.txt', binary=False)

Loading of formated fasttext file :

In [47]:
fasttext = KeyedVectors.load_word2vec_format(r'D:\COURS\A4\S8\Stage\Documents\Supervised-Learning-using-Unsupervised-Learning-Metrics-in-the-absence-of-Annotated-Data\custom_BERTScore\ftwords.txt')

In [ ]:
fasttext = KeyedVectors.load_word2vec_format(r'')

In [48]:
fasttext["house"]

array([ 0.75779974,  0.5959443 , -0.27542254,  0.24059881, -0.5490175 ,
        0.8300641 , -0.41410735,  0.7523287 ,  0.50460875, -0.86938876],
      dtype=float32)

##### Glove

Conversition to word2vec format (Do not compile unless necessary)

In [19]:
glove_path = r"D:\COURS\A4\S8\Stage\Documents\Supervised-Learning-using-Unsupervised-Learning-Metrics-in-the-absence-of-Annotated-Data\custom_BERTScore\glove.6B\glove.6B.300d.txt"
glove_temp = KeyedVectors.load_word2vec_format(glove_path, no_header=True)
glove_temp.save_word2vec_format(r"D:\COURS\A4\S8\Stage\Documents\Supervised-Learning-using-Unsupervised-Learning-Metrics-in-the-absence-of-Annotated-Data\custom_BERTScore\glove2word2vec.txt")

Loading of formated glove file :

In [24]:
glove = KeyedVectors.load_word2vec_format(r"D:\COURS\A4\S8\Stage\Documents\Supervised-Learning-using-Unsupervised-Learning-Metrics-in-the-absence-of-Annotated-Data\custom_BERTScore\glove2word2vec.txt")

### 2 - Similarity Measure

In [16]:
def SimilarityCandToRef(references, candidates):
    proximity = lambda x, y: (np.matmul(np.transpose(x), y))/(norm(x)*norm(y))

    all_proximities = []

    for candidate, reference in zip(candidates, references):
        proximities = []
        for c_word in candidate:
            sub_proximities = []
            for r_word in reference:
                sub_proximities.append(proximity(r_word, c_word))
            proximities.append(sub_proximities)
        all_proximities.append(proximities)
    return all_proximities

def SimilarityRefToCand(references, candidates):
    proximity = lambda x, y: (np.matmul(np.transpose(x), y))/(norm(x)*norm(y))

    all_proximities = []

    for candidate, reference in zip(candidates, references):
        proximities = []
        for r_word in reference:
            sub_proximities = []
            for c_word in candidate:
                sub_proximities.append(proximity(r_word, c_word))
            proximities.append(sub_proximities)
        all_proximities.append(proximities)
    return all_proximities

In [17]:
candToRef = SimilarityCandToRef(references, candidates)
refToCand = SimilarityRefToCand(references, candidates)
refToCand2 = np.transpose(candToRef) 

NameError: name 'references' is not defined

In [27]:
candToRef

[[[0.519005613856723, 0.18616442214576517, 0.04520518544899319],
  [0.09242192671658575, 0.012498354833030686, 0.025547976598766665],
  [0.2098326692090991, 0.3489852938028225, -0.00109822157119661],
  [0.0368609406903856, 0.020396980932637056, 1.0]]]

In [28]:
refToCand

[[[0.519005613856723,
   0.09242192671658575,
   0.2098326692090991,
   0.0368609406903856],
  [0.18616442214576517,
   0.012498354833030686,
   0.3489852938028225,
   0.020396980932637056],
  [0.04520518544899319, 0.025547976598766665, -0.00109822157119661, 1.0]]]

In [29]:
refToCand2

array([[[ 0.51900561],
        [ 0.09242193],
        [ 0.20983267],
        [ 0.03686094]],

       [[ 0.18616442],
        [ 0.01249835],
        [ 0.34898529],
        [ 0.02039698]],

       [[ 0.04520519],
        [ 0.02554798],
        [-0.00109822],
        [ 1.        ]]])

### 3 - Calculation of P, R and F

##### R calculation

In [44]:
fullSum = []
for individualSimilarity in candToRef:
    currentSum = 0
    for row in individualSimilarity:
        currentSum += row[np.argmax(row)]
    fullSum.append(currentSum)
R = []
for sum, reference in zip(fullSum, references):
    R.append((1/norm(reference))*sum)


In [45]:
R

[0.42425704559238325]

##### P Calculation

In [46]:
fullSum = []
for individualSimilarity in refToCand:
    currentSum = 0
    for row in individualSimilarity:
        currentSum += row[np.argmax(row)]
    fullSum.append(currentSum)
P = []
for sum, candidate in zip(fullSum, candidates):
    P.append((1/norm(candidate))*sum)

In [47]:
P

[0.3604158590807159]

##### F Calculation

In [48]:
F = []

for r, p in zip(R, P):
    f = 2*((p*r)/(p+r))
    F.append(f)

In [49]:
F

[0.38973938477441966]

##### Full Calculation

In [ ]:
def computeMetrics(candToRef, refToCand):
    # R computation
    fullSum = []
    for individualSimilarity in candToRef:
        currentSum = 0
        for row in individualSimilarity:
            currentSum += row[np.argmax(row)]
        fullSum.append(currentSum)
    R = []
    for sum, reference in zip(fullSum, references):
        R.append((1/norm(reference))*sum)

    # P computation
    fullSum = []
    for individualSimilarity in refToCand:
        currentSum = 0
        for row in individualSimilarity:
            currentSum += row[np.argmax(row)]
        fullSum.append(currentSum)
    P = []
    for sum, candidate in zip(fullSum, candidates):
        P.append((1/norm(candidate))*sum)
    
    # F computation
    F = []
    for r, p in zip(R, P):
        f = 2*((p*r)/(p+r))
        F.append(f)
    
    return (R, R, F)

### 4 - Basic Test of the package

In [28]:
from custom_score.score import score
from custom_score.utils import model_load

In [2]:
w2v = model_load("Word2Vec")

In [3]:
score(w2v)

([0.40998854847735094], [0.22880340017752968], [0.2937005518713847])

In [29]:
score(glove)

([0.03954665010154433], [0.05171590963278553], [0.04481993467824077])

## II - Billsum Dataset Test

### 0 - Utils

In [2]:
def DynamicEmbeddingSampleTest(data, limit=3, modelPath = None, model = None, nbLayers = 24):
    nbIter = 1
    scores = []
    init_time = datetime.now()
    for row in data.iterrows():
        curCand = [row[1][2]]
        curRef = row[1][1]
        curCand = [" ".join(row[1][2].split("\n"))]
        curRef = [" ".join(row[1][1].split("\n"))]
        assert len(curCand) == len(curRef)
        if modelPath != None:
            (P, R, F), hashname = bert_score.score(curCand, curRef, lang="en", 
                                        model_type=modelPath, 
                                        num_layers=nbLayers, return_hash=True)
        elif model != None:
            (P, R, F), hashname = bert_score.score(curCand, curRef, lang="en", 
                                        model_type=model, 
                                        num_layers=nbLayers, return_hash=True)
        else:
            (P, R, F), hashname = bert_score.score(curCand, curRef, lang="en", return_hash=True)
        P = P[0].item()
        R = R[0].item()
        F = F[0].item()
        scores.append((P, R, F))
        if nbIter >= limit:
            break
        nbIter += 1
    runtime = (datetime.now() - init_time).total_seconds()
    return scores, runtime

In [3]:
def StaticEmbeddingSampleTest(data, model, limit=3):
    nbIter = 1
    scores = []
    init_time = datetime.now()
    for row in data.iterrows():
        curCand = [row[1][2]]
        curRef = row[1][1]
        curCand = [" ".join(row[1][2].split("\n"))]
        curRef = [" ".join(row[1][1].split("\n"))]
        assert len(curCand) == len(curRef)
        
        (P, R, F) = score(model, curCand, curRef)
        P = P[0]
        R = R[0]
        F = F[0]
        scores.append((P, R, F))

        if nbIter >= limit:
            break
        nbIter += 1
    runtime = (datetime.now() - init_time).total_seconds()
    return scores, runtime
    

### 1 - Load Benchmarking Dataset

In [4]:
billsumPath = r"D:\COURS\A4\S8\Stage\Documents\Datasets\Summary Evaluation\BillSum\corpus"

In [37]:
#billsum_train = pd.read_json(path_or_buf = billsumPath + r"\us_train_data_final_OFFICIAL.jsonl", lines=True)
billsum_test = pd.read_json(path_or_buf = billsumPath + r"\us_test_data_final_OFFICIAL.jsonl", lines=True)

In [6]:
billsum_test.head(3)

,bill_id,text,summary,title,text_len,sum_len
0,110_hr37,SECTION 1. SHORT TITLE.\n\n This Act may be...,National Science Education Tax Incentive for B...,To amend the Internal Revenue Code of 1986 to ...,8494,321
1,112_hr2873,SECTION 1. SHORT TITLE.\n\n This Act may be...,Small Business Expansion and Hiring Act of 201...,To amend the Internal Revenue Code of 1986 to ...,6522,1424
2,109_s2408,SECTION 1. RELEASE OF DOCUMENTS CAPTURED IN IR...,Requires the Director of National Intelligence...,A bill to require the Director of National Int...,6154,463


### 2 - Test : RoBERTa Implementation

In [43]:
bert_scores, bert_runtime = DynamicEmbeddingSampleTest(billsum_test)

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaM

### 3 - Test : Word2Vec Implementation 

##### W2V Serialization

In [9]:
w2v = model_load("Word2Vec")

In [13]:
with open('serialized_w2v.pkl', 'wb') as f:
    pickle.dump(w2v, f)
    f.close()

##### W2V test

In [7]:
with open('serialized_w2v.pkl', 'rb') as f:
    w2v = pickle.load(f)
    f.close()

In [40]:
word2vec_scores, word2vec_runtime = StaticEmbeddingSampleTest(billsum_test, w2v, 3)

### 4 - Test : Fasttext Implementation 

### 5 - Test : Glove Implementation

In [9]:
glove = KeyedVectors.load_word2vec_format(r"D:\COURS\A4\S8\Stage\Documents\Supervised-Learning-using-Unsupervised-Learning-Metrics-in-the-absence-of-Annotated-Data\custom_BERTScore\glove2word2vec.txt")

In [10]:
glove_scores, glove_runtime = StaticEmbeddingSampleTest(billsum_test, glove)

### 6 - Evaluation

##### Runtime Evaluation

In [12]:
runtimeTable = [bert_runtime, word2vec_runtime, glove_runtime]
runtimeDf = pd.DataFrame(runtimeTable, columns=["runtime"], index=["Roberta-24-layers", "Word2Vec", "Glove"])
runtimeDf

,runtime
Roberta-24-layers,38.338537
Word2Vec,4.411479
Glove,4.890593


##### Quality Evaluation

In [44]:
bert_scores_Df = pd.DataFrame(bert_scores, columns=["Bert_P", "Bert_R", "Bert_F"])
bert_scores_Df

,Bert_P,Bert_R,Bert_F
0,0.874369,0.704338,0.780197
1,0.846002,0.728316,0.782760
2,0.873844,0.702725,0.778998


In [42]:
word2vec_scores_Df = pd.DataFrame(word2vec_scores, columns=["W2V_P", "W2V_R", "W2V_F"])
word2vec_scores_Df

,W2V_P,W2V_R,W2V_F
0,0.542677,0.904048,0.678230
1,0.769379,0.953081,0.851434
2,0.718532,0.909344,0.802755


In [20]:
billsum_test.head(10)

,bill_id,text,summary,title,text_len,sum_len
0,110_hr37,SECTION 1. SHORT TITLE.\n\n This Act may be...,National Science Education Tax Incentive for B...,To amend the Internal Revenue Code of 1986 to ...,8494,321
1,112_hr2873,SECTION 1. SHORT TITLE.\n\n This Act may be...,Small Business Expansion and Hiring Act of 201...,To amend the Internal Revenue Code of 1986 to ...,6522,1424
2,109_s2408,SECTION 1. RELEASE OF DOCUMENTS CAPTURED IN IR...,Requires the Director of National Intelligence...,A bill to require the Director of National Int...,6154,463
3,108_s1899,SECTION 1. SHORT TITLE.\n\n This Act may be...,National Cancer Act of 2003 - Amends the Publi...,A bill to improve data collection and dissemin...,19853,1400
4,107_s1531,SECTION 1. SHORT TITLE.\n\n This Act may be...,Military Call-up Relief Act - Amends the Inter...,A bill to amend the Internal Revenue Code of 1...,6273,278
5,107_hr4541,SECTION 1. RELIQUIDATION OF CERTAIN ENTRIES PR...,Requires the Customs Service to reliquidate ce...,To provide for reliquidation of entries premat...,11691,114
6,111_s1495,SECTION 1. SHORT TITLE.\n\n This Act may be...,Service Dogs for Veterans Act of 2009 - Direct...,A bill to require the Secretary of Veterans Af...,5328,379
7,111_s3885,SECTION 1. SHORT TITLE.\n\n This Act may be...,Race to the Top Act of 2010 - Directs the Secr...,A bill to provide incentives for States and lo...,16668,1525
8,113_hr1796,SECTION 1. SHORT TITLE.\n\n This Act may be...,Troop Talent Act of 2013 - Directs the Secreta...,Troop Talent Act of 2013,15352,2151
9,103_hr1987,SECTION 1. SHORT TITLE.\n\n This Act may be...,Taxpayer's Right to View Act of 1993 - Amends ...,Taxpayer's Right to View Act of 1993,5633,894


In [12]:
glove_scores_Df = pd.DataFrame(glove_scores, columns=["GLV_P", "GLV_R", "GLV_F"])
glove_scores_Df

,GLV_P,GLV_R,GLV_F
0,0.193759,-0.170188,-2.798021
1,1.496544,-0.422760,-1.178411
2,0.309721,-0.217137,-1.452770


# HOLDALL

### Padding

In [ ]:
# Padding
new_references = []
new_candidates = []

for reference, candidate in zip(references, candidates):
    size_diff = len(reference) - len(candidate)
    if size_diff >= 0:
        candidate = np.pad(candidate, (0, size_diff))
        reference = np.array(reference)
    else:
        reference = np.pad(reference, [(0, np.abs(size_diff)), (0, 0)], mode="constant")
        candidate = np.array(candidate)
    new_references.append(reference)
    new_candidates.append(candidate)
   

references = np.array(new_references, dtype=object)
candidates = np.array(new_candidates, dtype=object)

### Duplicating billsum summary column

In [ ]:
tab = []
for i in range(billsum_test.shape[0]):
    tab.append( billsum_test.iloc[i][2] + billsum_test.iloc[i][2])
billsum_test["summary"] = tab    